In [1]:
import jieba #裝繁體版本 https://github.com/APCLab/jieba-tw
import jieba.posseg as pseg
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter


In [2]:
# For RNN 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from keras.layers import Input, Concatenate, Embedding
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, History 
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
data = pd.read_csv("QS_1.csv") # 使用 pandas 讀取 CSV
query = data.to_numpy()
query = query[:, 1]
print(query)
print(query.shape)

for element in query:
    seg_list = jieba.cut(element)
    print(" / ".join(seg_list))
    
"""
通姦 / 在 / 刑法 / 上 / 應該 / 除罪化
應該 / 取消 / 機車 / 強制 / 二段式 / 左轉 / ( / 待轉 / )
支持 / 博弈 / 特區 / 在 / 台灣 / 合法化
中華 / 航空 / 空服員 / 罷工 / 是 / 合理 / 的
性交易 / 應該 / 合法化
ECFA / 早 / 收清單 / 可 / （ / 有 / ） / 達到 / 其 / 預期 / 成效
應該 / 減免 / 證所稅
贊成 / 中油 / 在 / 觀塘 / 興建 / 第三天 / 然氣 / 接收站
支持 / 中國 / 學生 / 納入 / 健保
支持 / 臺灣 / 中小學 / （ / 含 / 高職 / 、 / 專科 / ） / 服儀 / 規定 / （ / 含髮 / 、 / 襪 / 、 / 鞋 / ） / 給予 / 學生 / 自主
不 / 支持 / 使用 / 加密 / 貨幣
不 / 支持 / 學雜費 / 調漲
同意 / 政府 / 舉債 / 發展 / 前瞻 / 建設 / 計畫
支持 / 電競 / 列入 / 體育 / 競技
反對 / 台鐵 / 東移 / 徵收案
支持 / 陳前總統 / 保外 / 就醫
年金 / 改革 / 應 / 取消 / 或 / 應 / 調降 / 軍公教 / 月 / 退之 / 優存 / 利率 / 十八 / 趴
同意 / 動物 / 實驗
油價 / 應該 / 凍漲 / 或 / 緩漲
反對 / 旺旺中 / 時 / 併購 / 中嘉
"""

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\asus\AppData\Local\Temp\jieba.cache


['通姦在刑法上應該除罪化' '應該取消機車強制二段式左轉(待轉)' '支持博弈特區在台灣合法化' '中華航空空服員罷工是合理的'
 '性交易應該合法化' 'ECFA早收清單可（有）達到其預期成效' '應該減免證所稅' '贊成中油在觀塘興建第三天然氣接收站'
 '支持中國學生納入健保' '支持臺灣中小學（含高職、專科）服儀規定（含髮、襪、鞋）給予學生自主' '不支持使用加密貨幣' '不支持學雜費調漲'
 '同意政府舉債發展前瞻建設計畫' '支持電競列入體育競技' '反對台鐵東移徵收案' '支持陳前總統保外就醫'
 '年金改革應取消或應調降軍公教月退之優存利率十八趴' '同意動物實驗' '油價應該凍漲或緩漲' '反對旺旺中時併購中嘉']
(20,)


Loading model cost 0.847 seconds.
Prefix dict has been built succesfully.


通姦 / 在 / 刑法 / 上應 / 該 / 除罪 / 化
應該 / 取消 / 機車 / 強制 / 二段式 / 左轉 / ( / 待轉 / )
支持 / 博弈 / 特區 / 在 / 台灣 / 合法化
中華 / 航空 / 空服員 / 罷工 / 是 / 合理 / 的
性交易 / 應該 / 合法化
ECFA / 早 / 收清 / 單可 / （ / 有 / ） / 達到 / 其 / 預期 / 成效
應該 / 減免 / 證所稅
贊成 / 中油 / 在 / 觀塘 / 興建 / 第三 / 天然 / 氣 / 接收站
支持 / 中國學 / 生納入 / 健保
支持 / 臺 / 灣 / 中小 / 學 / （ / 含高職 / 、 / 專科 / ） / 服儀 / 規定 / （ / 含 / 髮 / 、 / 襪 / 、 / 鞋 / ） / 給予 / 學生 / 自主
不 / 支持 / 使用 / 加密 / 貨幣
不 / 支持 / 學雜費 / 調漲
同意 / 政府 / 舉債 / 發展 / 前瞻 / 建設 / 計畫
支持 / 電競 / 列入 / 體育 / 競技
反對 / 台 / 鐵東移 / 徵收 / 案
支持 / 陳 / 前 / 總統 / 保外 / 就醫
年 / 金 / 改革 / 應 / 取消 / 或應 / 調降 / 軍 / 公教 / 月 / 退 / 之 / 優存 / 利率 / 十八 / 趴
同意 / 動物 / 實驗
油價 / 應該 / 凍漲 / 或 / 緩漲
反對 / 旺旺 / 中時 / 併 / 購中 / 嘉


'\n通姦 / 在 / 刑法 / 上 / 應該 / 除罪化\n應該 / 取消 / 機車 / 強制 / 二段式 / 左轉 / ( / 待轉 / )\n支持 / 博弈 / 特區 / 在 / 台灣 / 合法化\n中華 / 航空 / 空服員 / 罷工 / 是 / 合理 / 的\n性交易 / 應該 / 合法化\nECFA / 早 / 收清單 / 可 / （ / 有 / ） / 達到 / 其 / 預期 / 成效\n應該 / 減免 / 證所稅\n贊成 / 中油 / 在 / 觀塘 / 興建 / 第三天 / 然氣 / 接收站\n支持 / 中國 / 學生 / 納入 / 健保\n支持 / 臺灣 / 中小學 / （ / 含 / 高職 / 、 / 專科 / ） / 服儀 / 規定 / （ / 含髮 / 、 / 襪 / 、 / 鞋 / ） / 給予 / 學生 / 自主\n不 / 支持 / 使用 / 加密 / 貨幣\n不 / 支持 / 學雜費 / 調漲\n同意 / 政府 / 舉債 / 發展 / 前瞻 / 建設 / 計畫\n支持 / 電競 / 列入 / 體育 / 競技\n反對 / 台鐵 / 東移 / 徵收案\n支持 / 陳前總統 / 保外 / 就醫\n年金 / 改革 / 應 / 取消 / 或 / 應 / 調降 / 軍公教 / 月 / 退之 / 優存 / 利率 / 十八 / 趴\n同意 / 動物 / 實驗\n油價 / 應該 / 凍漲 / 或 / 緩漲\n反對 / 旺旺中 / 時 / 併購 / 中嘉\n'

In [4]:
with open('../../NC_1_ALL.txt', 'r', encoding="utf-8") as f:
    AllNews = f.readlines()

print(len(AllNews))
print(AllNews[0][:12])
#print(type(AllNews[0][:12]))
print(query[0])
print(query[0][1])




print(AllNews[0][1:12] == "news_000001") #some cant seen word in begin of string
AllNews[0] = AllNews[0][1:]
print(AllNews[0][:11])
print(AllNews[0][14:])
seg_list = jieba.cut(AllNews[0][14:])
print(" / ".join(seg_list))

98250
﻿news_000001
通姦在刑法上應該除罪化
姦
True
news_000001
新北市第二選區議員候選人陳明義，爭取連任失利，成了落選頭，他認為選前清潔工事件影片遭人變造、剪接，影響選務人員公平性，將部分屬於他的選票認定為無效票或其他候選人的票，上月向新北地院聲請全面驗票，並對吊車尾當選的陳文治提出當選無效之訴。新北地院表示，日前確實收到陳明義的相關聲請，最快在今日就會分案，並擇期開庭，傳喚陳明義進一步說明，再認定是否有驗票必要。新北市第二選區議員候選人陳明義，去年九合一大新北市第二選區議員候選人陳明義，爭取連任失利，成了落選頭，他認為選前清潔工事件影片遭人變造、剪接，影響選務人員公平性，將部分屬於他的選票認定為無效票或其他候選人的票，上月向新北地院聲請全面驗票，並對吊車尾當選的陳文治提出當選無效之訴。新北地院表示，日前確實收到陳明義的相關聲請，最快在今日就會分案，並擇期開庭，傳喚陳明義進一步說明，再認定是否有驗票必要。新北市第二選區議員候選人陳明義，去年九合一大選前，因競選旗幟與布幕遭到清潔隊員拆除，他得知後怒嗆「讓你們掃個夠」，遭人錄下並上傳網路，引發社會輿論撻伐，更有上百名清潔工包圍陳明義競選總部。據了解，陳明義認為，該影片遭人剪接、變造，隨媒體強力放送，不僅重創他的形象，更影響選務人員的公平性，造成不少投給他的有效選票，遭選務人員認定為無效票，或是他人選票。陳明義指出，選務人員的不公，已構成《公職人員選舉罷免法》中的「當選票數不實，足認有影響選舉結果之虞」，向新北地院聲請對第二選區包括新莊、五股、林口與泰山共387個投開票所全面驗票。陳明義表示，上月委託2位律師研究，針對陳文治、張晉婷其中1人提出當選無效告訴，然因張已被檢方提出當選無效，才轉而對陳提出告訴；對於驗票與否，陳明義說，目前仍無法確認，全數細節仍待8日自上海返台後與律師討論。對此，市議員陳文治則說，「真金不怕火煉」，若要重新驗票就每個票匭逐一驗票，的確，票數贏的少，僅有15票之差，但也不會影響陳明義落選事實，何況，提出當選無效告訴、驗票與否，也是他的權利。

新北市 / 第二 / 選區 / 議員候 / 選人 / 陳 / 明義 / ， / 爭取 / 連任 / 失利 / ， / 成 / 了 / 落 / 選頭 / ， / 他 / 認為 / 選 / 前清 / 潔工 / 事件 / 影片 

In [5]:
totalWordBag = {}
counter = Counter()

#加入停用字中
stops = []
stops.append('\n') 
stops.append('\n\n')
stops.append('2011')
stops.append('-')
stops.append('〔')
stops.append('〕')
stops.append('，')
stops.append('「')
stops.append('」')
stops.append('。')
stops.append('，')
stops.append('；')
stops.append('？')
stops.append('：')
stops.append('『')
stops.append('』')
stops.append('(')
stops.append(')')
stops.append('、')
stops.append('！')
stops.append('\x08')
stops.append('（')
stops.append('）')
stops.append('\t')
stops.append('/')
stops.append(':')
stops.append('【')
stops.append('】')
stops.append('╱')
stops.append('')
stops.append('的')
stops.append('著')

newsCut = {}

for news in AllNews:
    terms = [t for t in jieba.cut(news[14:], cut_all=True) if t not in stops]
    counter.update(terms)
    newsCut[news[:11]] = terms

In [6]:
print(sum(counter.values()))
#result = sorted(counter.items(), key=lambda x:x[1], reverse=True)
result = list(counter.items())
print(result[1])
result = [item for item in result if item[1] > 1]
print(len(result))
#print(newsCut["news_000001"][0])

58678574
('北市', 50660)
94832


In [7]:
wordToIndex = {u[0]:i+1 for i, u in enumerate(result)} # ('北市', 50660) 的 (北市=>index)
#del counter
print(wordToIndex["北市"])

2


In [8]:
data = pd.read_csv("TD.csv") # 使用 pandas 讀取 CSV
print(data.head())
TD = data.to_numpy()
trainQueryX = []
trainNewsX = []
dataY = []

print(TD[0][1])
for each in TD:
    if each[1] in newsCut:
        seg_list = jieba.cut(each[0])
        _query = []
        
        _news = []
        for _word in seg_list:
            if _word in wordToIndex:
                _query.append( wordToIndex[ _word ] )
            else:
                _query.append(0)
        for i in range(len(newsCut[each[1]])):
            if newsCut[each[1]][i] in wordToIndex:
                _news.append( wordToIndex[ newsCut[each[1]][i] ] )
            else:
                _news.append(0)
        trainQueryX.append(_query)
        trainNewsX.append(_news)
        dataY.append(each[2])
    else:
        print( each[1] + " Not in txt")

        Query   News_Index  Relevance
0  支持陳前總統保外就醫  news_064209          2
1  支持陳前總統保外就醫  news_012873          1
2  支持陳前總統保外就醫  news_056765          3
3  支持陳前總統保外就醫  news_055305          1
4  支持陳前總統保外就醫  news_097573          3
news_064209
news_014437 Not in txt
news_042122 Not in txt
news_059972 Not in txt
news_013409 Not in txt
news_002482 Not in txt
news_003104 Not in txt
news_033917 Not in txt
news_071107 Not in txt
news_013721 Not in txt
news_060673 Not in txt
news_087071 Not in txt
news_019490 Not in txt
news_060908 Not in txt
news_039945 Not in txt
news_052382 Not in txt
news_043689 Not in txt
news_025079 Not in txt
news_087627 Not in txt
news_008873 Not in txt
news_084450 Not in txt
news_024997 Not in txt
news_039932 Not in txt
news_018139 Not in txt
news_019353 Not in txt
news_023689 Not in txt
news_002871 Not in txt
news_074771 Not in txt
news_093716 Not in txt
news_019620 Not in txt
news_047619 Not in txt
news_091442 Not in txt
news_009393 Not in txt
news_011881 Not in txt
n

In [9]:
#print(newsCut["news_097573"])
print(trainQueryX[4])
print(trainNewsX[4])

cMax = 0
for news in trainNewsX:
    if cMax < len(news):
        cMax = len(news)
print(cMax)

cMax = 0
for news in trainQueryX:
    if cMax < len(news):
        cMax = len(news)
print(cMax)

# one-hot encoding
trainY = to_categorical(dataY, num_classes=4) # 0 1 2 3
#y_test = to_categorical(y_test, num_classes=4)
print(trainY[4])

[1251, 10, 98, 0, 4605, 0]
[3606, 6478, 6479, 1203, 656, 168, 304, 14059, 4550, 841, 221, 521, 10, 11061, 794, 795, 171, 36, 139, 5210, 487, 2520, 1364, 3582, 1353, 504, 745, 1195, 2612, 114, 98, 138, 455, 10, 2026, 4596, 5203, 700, 652, 653, 4605, 77, 654, 14059, 4550, 841, 2811, 75, 3606, 2241, 166, 658, 168, 304, 10, 11061, 266, 1200, 284, 7, 10799, 10666, 280, 1665, 1287, 6, 521, 5100, 7837, 18, 238, 266, 6, 7, 183, 7456, 1892, 168, 103, 975, 264, 2814, 355, 934, 1132, 547, 280, 101, 8102, 58746, 7197, 1110, 5157, 355, 280, 2071, 628, 10, 11061, 62, 264, 746, 121, 3435, 28565, 1359, 3782, 1424, 89, 652, 653, 75, 3346, 1750, 126, 1347, 3803, 672, 1022, 6772, 7185, 2428, 1204, 2028, 628, 111, 1424, 103, 1200, 1101, 108, 78, 1306, 1894, 652, 653, 24, 940, 114, 3611, 940, 11662, 2523, 1366, 14059, 4550, 841, 111, 3606, 6478, 6479, 1203, 656, 168, 304, 14059, 4550, 841, 221, 521, 10, 11061, 794, 795, 171, 36, 139, 5210, 487, 2520, 1364, 3582, 1353, 504, 745, 1195, 2612, 114, 98, 138, 45

In [12]:
# 要 zero padding 成過定長度
#https://datascience.stackexchange.com/questions/26366/training-an-rnn-with-examples-of-different-lengths-in-keras

from keras.preprocessing.sequence import pad_sequences
#https://machinelearningmastery.com/data-preparation-variable-length-input-sequences-sequence-prediction/

# 16
trainQueryX = pad_sequences(trainQueryX, padding='post', maxlen=32)
print(trainQueryX.shape)

#4153
trainNewsX = pad_sequences(trainNewsX, padding='post', maxlen=5120)
print(trainNewsX.shape)
#trainQueryX = np.reshape(trainQueryX, (trainQueryX.shape[0], trainQueryX.shape[1], 1))
#trainNewsX = np.reshape(trainNewsX, (trainNewsX.shape[0], trainNewsX.shape[1], 1))

# 總共有 4678 筆，每筆不同長度，每次一筆(每個字視為一個 feature) -> 每筆回傳一個 0~3 的判斷


from sklearn.model_selection import train_test_split
X_train_Query, X_test_Query, y_train, y_test = train_test_split(trainQueryX, trainY, test_size=0.3, random_state=41)
X_train_News, X_test_News, y_train, y_test = train_test_split(trainNewsX, trainY, test_size=0.3, random_state=41)

(4678, 32)
(4678, 5120)


In [13]:
#input (query, documents) output 0~3
#https://blog.csdn.net/churximi/article/details/61210129

def rnn(vocab_size, embedding_dim, seq_len=1000):
    #(4678, 不定長度)
    inputA = Input(shape=(32, )) # 假設 1000 個詞(zero padding)
    x = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputA)
    inputB = Input(shape=(5120, ))
    y = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputB)
    #本來 LSTM 吃的 input: (N 個, 每個長度, 每個 feature 數量)
    #這邊是把 (N, 32) -> embedding -> (N, 32, embedding_dim)
    #有種 [0][0] = 1 -> [0][0] = (1, 1, 1, 1, ...., 1)
    #不然本來要 [0][0] = 1 -> [0][0][0] = 1
    
    
    
    # the first branch operates on the first input
    #(batch_size, timesteps, input_dim)
    x = LSTM(32)(x)
    
    # the second branch opreates on the second input
    y = LSTM(256)(y)
    
    # combine the output of the two branches
    combined = Concatenate()([x, y])

    # apply a FC layer and then a regression prediction on the
    # combined outputs
    z = Dense(32, activation="relu")(combined)
    z = Dense(4, activation="sigmoid")(z)
    
    # our model will accept the inputs of the two branches and
    # then output a single value
    model = Model(inputs=[inputA, inputB], outputs=z)
    return model


model = rnn(len(result)+1, 128)
print(model.summary(90))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

__________________________________________________________________________________________
Layer (type)                 Output Shape        Param #    Connected to                  
input_3 (InputLayer)         (None, 32)          0                                        
__________________________________________________________________________________________
input_4 (InputLayer)         (None, 5120)        0                                        
__________________________________________________________________________________________
embedding_3 (Embedding)      (None, 32, 128)     12138624   input_3[0][0]                 
__________________________________________________________________________________________
embedding_4 (Embedding)      (None, 5120, 128)   12138624   input_4[0][0]                 
__________________________________________________________________________________________
lstm_3 (LSTM)                (None, 32)          20608      embedding_3[0][0]             

In [14]:
print("[INFO] training model...")

#https://keras.io/zh/callbacks/
#keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
modelCheck = ModelCheckpoint('LSTM{epoch:08d}.h5', save_weights_only=True, save_best_only=True)
#keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
earlyStop = EarlyStopping(monitor='val_loss', patience=50)
history = History()

model.fit([X_train_Query, X_train_News], y_train, 
          batch_size=32,
          epochs=10,
          validation_data=([X_test_Query, X_test_News], y_test),
          callbacks=[modelCheck, earlyStop, history])
    #,validation_data=([testAttrX, testImagesX], testY)

[INFO] training model...
Instructions for updating:
Use tf.cast instead.
Train on 3274 samples, validate on 1404 samples
Epoch 1/10
3274/3274 [==============================] - 1566s 478ms/step - loss: 1.2121 - acc: 0.4435 - val_loss: 1.1173 - val_acc: 0.5064
Epoch 2/10
3274/3274 [==============================] - 1678s 513ms/step - loss: 1.0671 - acc: 0.5342 - val_loss: 1.2425 - val_acc: 0.2472
Epoch 3/10
3274/3274 [==============================] - 1567s 479ms/step - loss: 1.1698 - acc: 0.4551 - val_loss: 1.1431 - val_acc: 0.5128
Epoch 4/10
3274/3274 [==============================] - 1509s 461ms/step - loss: 0.9116 - acc: 0.5977 - val_loss: 1.2369 - val_acc: 0.4793
Epoch 5/10
3274/3274 [==============================] - 1505s 460ms/step - loss: 0.7001 - acc: 0.7193 - val_loss: 1.4533 - val_acc: 0.4466
Epoch 6/10
3274/3274 [==============================] - 1506s 460ms/step - loss: 0.5251 - acc: 0.8033 - val_loss: 1.5683 - val_acc: 0.4523
Epoch 7/10
3274/3274 [=======================

In [15]:
print("[INFO] training model...")

#https://keras.io/zh/callbacks/
#keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
modelCheck = ModelCheckpoint('LSTM_NO_VAL{epoch:08d}.h5', save_weights_only=True, save_best_only=True)
#keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
earlyStop = EarlyStopping(monitor='val_loss', patience=50)
history = History()

model.fit([trainQueryX, trainNewsX], trainY, 
          batch_size=32,
          epochs=10,
          callbacks=[modelCheck, history])
    #,validation_data=([testAttrX, testImagesX], testY)

[INFO] training model...
Epoch 1/10
4678/4678 [==============================] - 2043s 437ms/step - loss: 0.6294 - acc: 0.8053
Epoch 2/10


C:\Users\asus\Miniconda3\envs\testJ\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


4678/4678 [==============================] - 2028s 434ms/step - loss: 0.3891 - acc: 0.8775
Epoch 3/10
4678/4678 [==============================] - 1906s 407ms/step - loss: 0.2631 - acc: 0.9196
Epoch 4/10
4678/4678 [==============================] - 1905s 407ms/step - loss: 0.1911 - acc: 0.9389
Epoch 5/10
4678/4678 [==============================] - 1906s 407ms/step - loss: 0.1468 - acc: 0.9547
Epoch 6/10
4678/4678 [==============================] - 1908s 408ms/step - loss: 0.1211 - acc: 0.9617
Epoch 7/10
4678/4678 [==============================] - 1909s 408ms/step - loss: 0.1009 - acc: 0.9669
Epoch 8/10
4678/4678 [==============================] - 1913s 409ms/step - loss: 0.0748 - acc: 0.9741
Epoch 9/10
4678/4678 [==============================] - 1906s 407ms/step - loss: 0.0584 - acc: 0.9820
Epoch 10/10
4678/4678 [==============================] - 1964s 420ms/step - loss: 0.0488 - acc: 0.9835


In [21]:
#fully process
#turn all news to vector

testNewsX = []

print(TD[0][1])
for news in AllNews:
    if news[:11] in newsCut:
        _news = []
        for i in range(len(newsCut[news[:11]])):
            if newsCut[news[:11]][i] in wordToIndex:
                _news.append( wordToIndex[ newsCut[news[:11]][i] ] )
            else:
                _news.append(0)
        if(len(_news) > 5120):
            print(news[:11])
            print(len(_news))
        else:
            testNewsX.append(_news)
    else:
        print( each[1] + " Not in txt")



news_064209
6442
6890
5275
6453
5761
6837
11578
5263
5997
7057
7275
7733
8231
6507
6663
7514
5365
6492
5137
9075
8169
5279
10428
7585
5208
5145
7580
6969
6130
8122
5122
5386
5786
6366
5766
6867
5255
11576
5199
9563
8715
5236
7574


In [19]:
testMax = 0
for element in testNewsX:
    if(len(element) > testMax):
        testMax = len(element)
    if(len(element) > 5120):
        print(len(element))
print(testMax)

6442
6890
5275
6453
5761
6837
11578
5263
5997
7057
7275
7733
8231
6507
6663
7514
5365
6492
5137
9075
8169
5279
10428
7585
5208
5145
7580
6969
6130
8122
5122
5386
5786
6366
5766
6867
5255
11576
5199
9563
8715
5236
7574
11578


In [22]:
#turn query to vector
allQueryCut = []   
for element in query:
    seg_list = jieba.cut(element)
    _query = []
    for _word in seg_list:
        if _word in wordToIndex:
            _query.append( wordToIndex[ _word ] )
        else:
            _query.append(0)
    allQueryCut.append(_query)

In [25]:
allQueryCut = np.array(allQueryCut)
print(len(allQueryCut))
print(allQueryCut[0])
print(len(testNewsX))

20
[0, 75, 9726, 0, 142, 10038, 649]
98207


In [33]:
#把每一筆抓出來建立整個長度的複製
query_1_X = np.array([allQueryCut[0]])
query_1_X = np.repeat(query_1_X, [98207], axis=0)
print(query_1_X)

[[    0    75  9726 ...   142 10038   649]
 [    0    75  9726 ...   142 10038   649]
 [    0    75  9726 ...   142 10038   649]
 ...
 [    0    75  9726 ...   142 10038   649]
 [    0    75  9726 ...   142 10038   649]
 [    0    75  9726 ...   142 10038   649]]


In [35]:
query_1_X = pad_sequences(query_1_X, padding='post', maxlen=32)
testNewsX = pad_sequences(testNewsX, padding='post', maxlen=5120)

q1result = model.predict([query_1_X, testNewsX], verbose=1)

98207/98207 [==============================] - 13147s 134ms/step


In [47]:
print(q1result[:10])

[[4.4053793e-04 8.8502055e-01 7.3882937e-04 2.3797154e-04]
 [2.3129582e-04 1.9631457e-01 3.2583773e-03 1.3151765e-04]
 [2.8210163e-02 3.6771542e-01 6.0207069e-02 2.4238527e-03]
 [1.7073426e-01 3.8785994e-02 2.4257302e-02 3.5274029e-04]
 [6.3630939e-04 9.8658413e-02 3.4915179e-02 1.1253953e-03]
 [6.8247318e-06 7.5489283e-05 2.3558736e-04 8.5958862e-01]
 [9.3373060e-03 4.9413294e-02 3.1966120e-02 8.7706149e-03]
 [1.0311604e-05 2.4232355e-01 1.5037954e-03 3.3599138e-04]
 [2.9221177e-04 4.5322984e-01 6.4130127e-03 1.5336275e-03]
 [6.6279471e-03 4.1341269e-01 3.2921731e-03 1.8486381e-04]]


In [70]:
indexToNews = []
for news in AllNews:
    if news[:11] in newsCut:
        _news = []
        for i in range(len(newsCut[news[:11]])):
            if newsCut[news[:11]][i] in wordToIndex:
                _news.append( wordToIndex[ newsCut[news[:11]][i] ] )
            else:
                _news.append(0)
        if(len(_news) > 5120):
            print(news[:11])
            print(len(_news))
        else:
            indexToNews.append(news[:11])

news_000493
6442
news_001830
6890
news_005859
5275
news_006624
6453
news_009760
5761
news_011974
6837
news_013296
11578
news_013986
5263
news_015160
5997
news_018618
7057
news_019024
7275
news_021565
7733
news_022512
8231
news_022617
6507
news_027638
6663
news_031533
7514
news_031565
5365
news_036580
6492
news_038126
5137
news_041235
9075
news_044698
8169
news_048946
5279
news_050600
10428
news_051081
7585
news_055783
5208
news_064355
5145
news_065503
7580
news_068718
6969
news_071636
6130
news_074446
8122
news_075524
5122
news_079978
5386
news_080177
5786
news_082890
6366
news_085389
5766
news_086699
6867
news_089666
5255
news_089781
11576
news_096070
5199
news_096395
9563
news_098508
8715
news_099607
5236
news_099764
7574


In [83]:
q1_1_count = 0
Q1result = []
for i in range(len(q1result)):
    if q1result[i][3] > 0.915:
        q1_1_count += 1
        Q1result.append([indexToNews[i], q1result[i][3]])
print(q1_1_count)
print(Q1result[:5])



315
[['news_000588', 0.97795266], ['news_000686', 0.9706557], ['news_000801', 0.9817699], ['news_000892', 0.9466374], ['news_000972', 0.9801428]]


In [86]:
import csv

with open('query1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['新聞', '機率'])
    for word, num in Q1result:
        writer.writerow([word, num])


with open('query1total2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['新聞', '機率0', '機率1', '機率2', '機率3'])
    for i in range(len(q1result)):
        writer.writerow([indexToNews[i], q1result[i][0], q1result[i][1], q1result[i][2], q1result[i][3]])

In [63]:
model2 = rnn(len(result)+1, 128)
model2.load_weights("LSTM00000001.h5")
q1resultv2 = model2.predict([query_1_X, testNewsX], verbose=1)

98207/98207 [==============================] - 12465s 127ms/step


In [73]:
q1_2_count = 0
Q1resultv2 = []
for i in range(len(q1resultv2)):
    if q1resultv2[i][3] > 0.22:
        q1_2_count += 1
        Q1resultv2.append([indexToNews[i], q1resultv2[i][3]])
print(q1_2_count)
print(Q1resultv2[:5])

553
[['news_000058', 0.25337654], ['news_000402', 0.2396459], ['news_000485', 0.23715794], ['news_000960', 0.22012448], ['news_001409', 0.26034182]]


In [ ]:
# 發現長度要 12288 才夠

def rnn(vocab_size, embedding_dim, seq_len=1000):
    #(4678, 不定長度)
    inputA = Input(shape=(32, )) # 假設 1000 個詞(zero padding)
    x = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputA)
    inputB = Input(shape=(12288, ))
    y = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputB)
    #本來 LSTM 吃的 input: (N 個, 每個長度, 每個 feature 數量)
    #這邊是把 (N, 32) -> embedding -> (N, 32, embedding_dim)
    #有種 [0][0] = 1 -> [0][0] = (1, 1, 1, 1, ...., 1)
    #不然本來要 [0][0] = 1 -> [0][0][0] = 1
    
    
    
    # the first branch operates on the first input
    #(batch_size, timesteps, input_dim)
    x = LSTM(32)(x)
    
    # the second branch opreates on the second input
    y = LSTM(256)(y)
    
    # combine the output of the two branches
    combined = Concatenate()([x, y])

    # apply a FC layer and then a regression prediction on the
    # combined outputs
    z = Dense(32, activation="relu")(combined)
    z = Dense(4, activation="sigmoid")(z)
    
    # our model will accept the inputs of the two branches and
    # then output a single value
    model = Model(inputs=[inputA, inputB], outputs=z)
    return model


model = rnn(len(result)+1, 128)
print(model.summary(90))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.preprocessing.sequence import pad_sequences

# 16
trainQueryX = pad_sequences(trainQueryX, padding='post', maxlen=32)
print(trainQueryX.shape)

#12288
trainNewsX = pad_sequences(trainNewsX, padding='post', maxlen=12288)
print(trainNewsX.shape)

from sklearn.model_selection import train_test_split
X_train_Query, X_test_Query, y_train, y_test = train_test_split(trainQueryX, trainY, test_size=0.3, random_state=41)
X_train_News, X_test_News, y_train, y_test = train_test_split(trainNewsX, trainY, test_size=0.3, random_state=41)

In [ ]:
print("[INFO] training model...")
modelCheck = ModelCheckpoint('LSTM12288_{epoch:02d}.h5', save_weights_only=True, save_best_only=True)
history = History()

model.fit([X_train_Query, X_train_News], y_train, 
          batch_size=64,
          epochs=10,
          validation_data=([X_test_Query, X_test_News], y_test),
          callbacks=[modelCheck, history])


In [46]:
Q1vectors = np.zeros((230,11790), dtype=np.int)
totalWordBagArray = list(totalWordBag.keys())
for i in range(len(newsList)):
    newsDict = newsList[i]
    for j in range(len(totalWordBagArray) ):
        if totalWordBagArray[j] in newsDict:
            Q1vectors[i][j] = 1

with np.printoptions(threshold=np.inf):
    print(Q1vectors[229])
    
if True:
    np.savetxt("Q1vectors.csv", Q1vectors, delimiter=",")

[0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 1 0 0 0
 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
import pickle

def save_obj(obj, name):
    with open("./" + name + ".pkl", "wb") as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name):
    with open("./" + name + ".pkl", "rb") as f:
        return pickle.load(f)
    
save_obj(totalWordBag, "Q1wordBag")

In [13]:
import csv

with open('query1.txt', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['詞彙', '出現次數'])
    for word, num in result[:100]:
        writer.writerow([word, num])

NameError: name 'result' is not defined

In [13]:
model = Sequential()
model.add(Embedding(1000, 64, input_length=10))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be
# no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

input_array = np.random.randint(1000, size=(32, 10))
# [32*10]
print(input_array.shape)

model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
assert output_array.shape == (32, 10, 64)
print(output_array.shape)

Instructions for updating:
Colocations handled automatically by placer.
(32, 10)
(32, 10, 64)


In [17]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical

"""
model = Sequential()

model.add(LSTM(32, return_sequences=True, input_shape=(None, 5)))
model.add(LSTM(8, return_sequences=True))
model.add(TimeDistributed(Dense(2, activation='sigmoid')))
"""

inputA = Input(shape=(None, 5)) # 假設 1000 個詞(zero padding)
x = LSTM(32)(inputA)
#x = LSTM(8, return_sequences=True)(x)
x = TimeDistributed(Dense(2, activation='sigmoid'))(x)
model = Model(inputs=inputA, outputs=x)

print(model.summary(100))

model.compile(loss='categorical_crossentropy',
              optimizer='adam')

def train_generator():
    while True:
        sequence_length = np.random.randint(10, 100) # 長度隨機介於 10~100
        print(sequence_length)
        x_train = np.random.random((1000, sequence_length, 5))# [1000][固定長度][5]
        # y_train will depend on past 5 timesteps of x
        y_train = x_train[:, :, 0]
        for i in range(1, 5):
            y_train[:, i:] += x_train[:, :-i, i]
        y_train = to_categorical(y_train > 2.5)
        yield x_train, y_train

#model.fit_generator(train_generator(), steps_per_epoch=30, epochs=10, verbose=1)


Instructions for updating:
Colocations handled automatically by placer.


AssertionError: 